## Data Loading

In [1]:
import numpy as np
import pandas as pd
churn = pd.read_csv('dataset.csv', sep=',')

## Data Analysis

In [2]:
data_size = churn.shape
print(data_size)

(3333, 21)


In [3]:
churn_col_names = list(churn.columns)
print(churn_col_names)

['state', 'account length', 'area code', 'phone number', 'international plan', 'voice mail plan', 'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls', 'churn']
